# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [49]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [50]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.09,96,100,20.62,TF,1683587099
1,1,kailua-kona,19.6406,-155.9956,28.98,76,40,5.14,US,1683586586
2,2,gao,16.6362,1.6370,32.42,9,65,4.05,ML,1683586558
3,3,ushuaia,-54.8000,-68.3000,8.81,57,75,5.14,AR,1683586503
4,4,perry,37.2506,-83.1999,25.10,91,100,2.21,US,1683587100


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:
#%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['City', 'Humidity'],
    alpha = 0.5

)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_locations = city_data_df.loc[(city_data_df['Max Temp'] < 27) & 
                       (city_data_df['Max Temp'] > 21) & 
                       (city_data_df['Wind Speed'] < 4.5) &
                       (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values


# Display sample data
ideal_locations

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,gonabad,34.3529,58.6837,21.08,20,0,2.22,IR,1683587100
37,37,duba,27.3513,35.6901,23.61,48,0,0.73,SA,1683586545
44,44,bethel,41.3712,-73.4140,22.20,29,0,2.57,US,1683587106
128,128,tamanrasset,22.7850,5.5228,25.95,11,0,2.06,DZ,1683586524
155,155,tura,25.5198,90.2201,22.79,38,0,2.11,IN,1683586549
181,181,badarpur,24.9000,92.6000,23.21,59,0,1.45,IN,1683587125
322,322,brookings,44.3114,-96.7984,22.92,47,0,2.06,US,1683587162
334,334,sohag,26.5570,31.6948,22.16,36,0,3.13,EG,1683587164
461,461,kill devil hills,36.0307,-75.6760,26.00,59,0,3.09,US,1683586885
506,506,al kharijah,25.4514,30.5464,21.32,31,0,4.14,EG,1683587189


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_locations[['City', 'Country', 'Latitude', 'Longitude', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df_index_reset = hotel_df.reset_index(drop=True)
hotel_df_index_reset

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,gonabad,IR,34.3529,58.6837,20,
1,duba,SA,27.3513,35.6901,48,
2,bethel,US,41.3712,-73.4140,29,
3,tamanrasset,DZ,22.7850,5.5228,11,
4,tura,IN,25.5198,90.2201,38,
5,badarpur,IN,24.9000,92.6000,59,
6,brookings,US,44.3114,-96.7984,47,
7,sohag,EG,26.5570,31.6948,36,
8,kill devil hills,US,36.0307,-75.6760,59,
9,al kharijah,EG,25.4514,30.5464,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {
    'categories': categories,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df_index_reset.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df_index_reset.loc[index, 'Longitude']
    latitude = hotel_df_index_reset.loc[index, 'Latitude']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df_index_reset.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df_index_reset.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df_index_reset.loc[index, 'City']} - nearest hotel: {hotel_df_index_reset.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df_index_reset

Starting hotel search
gonabad - nearest hotel: هتل پاسارگاد
duba - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
tamanrasset - nearest hotel: فندق الأمان
tura - nearest hotel: No hotel found
badarpur - nearest hotel: Meera Lodge
brookings - nearest hotel: Quality Inn
sohag - nearest hotel: Al Safa Hotel
kill devil hills - nearest hotel: Mariner Days Inn & Suites
al kharijah - nearest hotel: Kharga Hotel
ambasa - nearest hotel: No hotel found


,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,gonabad,IR,34.3529,58.6837,20,هتل پاسارگاد
1,duba,SA,27.3513,35.6901,48,No hotel found
2,bethel,US,41.3712,-73.4140,29,Hampton Inn Danbury
3,tamanrasset,DZ,22.7850,5.5228,11,فندق الأمان
4,tura,IN,25.5198,90.2201,38,No hotel found
5,badarpur,IN,24.9000,92.6000,59,Meera Lodge
6,brookings,US,44.3114,-96.7984,47,Quality Inn
7,sohag,EG,26.5570,31.6948,36,Al Safa Hotel
8,kill devil hills,US,36.0307,-75.6760,59,Mariner Days Inn & Suites
9,al kharijah,EG,25.4514,30.5464,31,Kharga Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
# %%capture --no-display

# Configure the map plot
hotel_map = hotel_df_index_reset.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['City', 'Humidity', 'Hotel Name', 'Country'],
    alpha = 0.5

)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name,Country)